## MongoDB Atlas Quickstart

[MongoDB Atlas Vector Search](https://www.mongodb.com/products/platform/atlas-vector-search) is part of the MongoDB platform that enables MongoDB customers to build intelligent applications powered by semantic search over any type of data. Atlas Vector Search allows you to integrate your operational database and vector search in a single, unified, fully managed platform with full vector database capabilities.

You can integrate TruLens with your application built on Atlas Vector Search to leverage observability and measure improvements in your application's search capabilities.

This tutorial will walk you through the process of setting up TruLens with MongoDB Atlas Vector Search and Llama-Index as the orchestrator.

Even better, you'll learn how to use metadata filters to create specialized query engines and leverage a router to choose the most appropriate query engine based on the query.

See [MongoDB Atlas/LlamaIndex Quickstart](https://www.mongodb.com/docs/atlas/atlas-vector-search/ai-integrations/llamaindex/) for more details.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/vector-dbs/mongodb_atlas/atlas_quickstart.ipynb)



In [2]:
!pip install pymongo[srv]
!pip install llama-index
!pip install llama-index-core
!pip install llama-index-vector-stores-mongodb
!pip install llama-index-embeddings-openai
!pip install llama-index-readers-mongodb
!pip install matplotlib
!pip install datasets
!pip install pandas
!pip install trulens-eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

## Import TruLens and start the dashboard

In [3]:
from trulens_eval import Tru

tru = Tru()

tru.reset_database()

tru.run_dashboard()

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.
Starting dashboard ...
npx: installed 22 in 4.328s

Go to this url and submit the ip given here. your url is: https://grumpy-rings-fall.loca.lt

  Submit this IP Address: 34.173.189.246



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Set imports, keys and llama-index settings

In [4]:
import getpass, os, pymongo, pprint
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.settings import Settings
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, ExactMatchFilter, FilterOperator
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

## Create a vector store

Next you need to create an Atlas Vector Search Index.

When you do so, use the following in the json editor:

```
{
  "fields": [
    {
      "numDimensions": 1536,
      "path": "embedding",
      "similarity": "cosine",
      "type": "vector"
    },
    {
      "path": "metadata.file_name",
      "type": "filter"
    }
  ]
}
```

In [5]:
from os import environ
from datasets import load_dataset
import pandas as pd
import json
import pprint
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.settings import Settings
from llama_index.core import Document
from llama_index.core.schema import MetadataMode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.response.notebook_utils import display_response
import pymongo


MONGO_URI = your-mongo-url
os.environ["OPENAI_API_KEY"] = your-api-key

embed_model = OpenAIEmbedding(model="text-embedding-3-small", dimensions=256)
llm = OpenAI()
Settings.llm = llm
Settings.embed_model = embed_model


if not MONGO_URI:
    print("MONGO_URI not set in environment variables")


def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None


mongo_client = get_mongo_client(MONGO_URI)

DB_NAME = "itdata"
COLLECTION_NAME = "it_support_data"

db = mongo_client[DB_NAME]
collection = db[COLLECTION_NAME]


vector_store = MongoDBAtlasVectorSearch(mongo_client, db_name=DB_NAME, collection_name=COLLECTION_NAME, index_name="vector_index")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents([], storage_context=storage_context)

query_engine = index.as_query_engine()

Connection to MongoDB successful


## Setup basic RAG

In [5]:
def generate_embedding(text):
    return embed_model.get_text_embedding(text)

def getAnswer(question):
    results = collection.aggregate(
        [
            {
                "$vectorSearch": {
                    "queryVector": generate_embedding(question),
                    "path": "embedding",
                    "numCandidates": 100,
                    "limit": 3,
                    "index": "vector_index",
                }
            }
        ]
    )

    results = list(results)

    if len(results) == 0:
        return "Sorry, I don't have an answer for that question."
    firstDocText = results[0]['text'].strip('')
    return  firstDocText

In [6]:
getAnswer("How to get BU account?")

'What is a BU Web Account? A BU Web Account is an online account that individuals who are applying to Boston University academic programs, planning to register for online learning programs, or planning to utilize select BU resources will use to maintain online materials, monitor status updates, and gain access to online registration.'

#Trulens Standalone Recorder Feedback Functions
###(For custom query function e.g. mongo collections aggregate, summarization using GPT-4)

In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI

import numpy as np

provider = OpenAI()

grounded = Groundedness(groundedness_provider=provider)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

# Query Engine Feedback Functions

In [6]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval.feedback import Groundedness
from trulens_eval import Feedback
from trulens_eval.app import App
import numpy as np

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.

context = App.select_context(query_engine)
grounded = Groundedness(groundedness_provider=provider)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on_input_output()
)
# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [7]:
from trulens_eval.generate_test_set import GenerateTestSet

test_questions = []
first50docs = collection.find().limit(50)
for doc in first50docs:
    question = doc['metadata']['question']
    test_questions.append(question)

test_set_50_bu_questions = {'50 Boston University IT Support Questions': test_questions}
print(test_set_50_bu_questions)

# Random questions someone might ask to a chat bot
random_chatbot_questions=[
    "Hello",
    "How are you?",
    "What is your task?",
    "What's the weather like today?",
    "How's your day going?",
    "Can you tell me a joke?",
    "What's the meaning of life?",
    "Do you know any interesting facts?",
    "Who is your favorite superhero?",
    "You are an asshole"
]

test_set_random_chatbot_questions = {'Unrelated Questions for which there is no data': random_chatbot_questions}

{'50 Boston University IT Support Questions': ['"What is the newest learning management system used at Boston University?"', '"Does Boston University offer a laptop loan program for students?"', '"What is Blackboard Learn?"', '"What are the key features of Blackboard Learn?"', '"What to expect from Blackboard Learn?"', '"How do I get started with Blackboard Learn?"', '"What does the IT Help Center provide?"', '"How do I contact the IT Help Center?"', '"What services does the IT Help Center offer?"', '"Do I need a current BU login name to use some services?"', '"How much do hardware repair and premium services cost?"', '"Can you help me with something?"', '"How do I enter my question or request?"', '"Do you provide responses via email?"', '"How do you request AV/IT equipment in a Boston University classroom?"', '"What kind of AV/IT equipment is available for reservation in Boston University classrooms?"', '"How do I request repair for AV/IT equipment in a Boston University classroom?"',

##Using Query Engine Recorder using Llama Index Query Engine for querying


In [8]:
from trulens_eval import TruLlama
tru_query_engine_recorder = TruLlama(query_engine,
    app_id='Basic RAG',
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance])

## Using Standalone Recorder:Mongo collections.aggregate(), GPT-4 Summarization for querying


In [ ]:
from trulens_eval import TruBasicApp
tru_llm_standalone_recorder = TruBasicApp(
    getAnswer, app_id="Happy Bot", feedbacks=[f_answer_relevance]
)


In [ ]:
test_prompts = test_set_random_chatbot_questions['Unrelated Questions for which there is no data']
with tru_llm_standalone_recorder as recording:
    for i in range(len(test_prompts)):
        test_prompt = test_prompts[i]
        tru_llm_standalone_recorder.app(test_prompt)


## Write test cases

Let's write a few test queries to test the ability of our RAG to answer questions on both documents in the vector store.

## Get testing!

Our test set is made up of 2 topics (test breadth), each with 2-3 questions (test depth).

We can store the topic as record level metadata and then test queries from each topic, using `tru_query_engine_recorder` as a context manager.

In [9]:
with tru_query_engine_recorder as recording:
    for category in test_set_random_chatbot_questions:
        recording.record_metadata=dict(prompt_category=category)
        test_prompts = test_set_random_chatbot_questions[category]
        for test_prompt in test_prompts:
            response = query_engine.query(test_prompt)

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

## Check evaluation results

Evaluation results can be viewed in the TruLens dashboard (started at the top of the notebook) or directly in the notebook.

In [11]:
tru.get_leaderboard()

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
Basic RAG,0.825532,0.941667,0.774,2.58,0.000611


In [18]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Submit this IP Address: 34.73.44.152



<Popen: returncode: 0 args: ['streamlit', 'run', '--server.headless=True', '...>